## IRI Data - Q3

Problem #3

The IRI data is set available on the AWS instance.  Several additional documents describing the data set and pointing to additional Bibliography are made available (including the white paper: Bronnenberg, Bart J., Kruger, Michael W, and Mela, Carl F. The IRI Marketing Dataset.). <br /> 
<br />
Getting to know this data set is time well spent as it will be very applicable towards the Fina project.  The data set is of medium size and complexity – however understanding the data and how file/folders/datasets related to each other will be of crucial importance for creating an appropriate training data set.  <br />
<br />
The data set has been purchase by UCSD, it is only for internal use and comes with restrictions.  Please read and follow the NDA specified at the very end of the “IRI_database_technical_appendix” document provided with the data set. <br />
<br /> 
I order to answer the questions below – document and describe what kind of data preparation and cleaning techniques did you have to apply?  What kinds of anomalies and missing data did you encounter?  Document how what assumption you made and how you approach the issues.
1. What are the most popular item(s) for a chosen product/category?<br />
2. How does the popularity of an item change with the cost associated with it for a chosen year?<br />
3. How is the popularity affected over the years? <br />
<br />
Extra credit: 
Apply K-nearest neighbor algorithms to a chose subset of data for a specific set of products/categories. 

In [1]:
import pandas as pd
import numpy as np

In [2]:
## Get data for Salty snacks
%cd /Users/Deepthi/Documents/DSE/Q3_DSE220 - Machine Learning/IRI Data - Project/Data

/Users/Deepthi/Documents/DSE/Q3_DSE220 - Machine Learning/IRI Data - Project/Data


#### Get data

##### Store level weekly data

In [3]:
# Considering only 1 year for the analysis
store_data = {}
# yrs = ['09','10','11']
# yrs = ['01','02']
yrs = ['11']
import os
for i in yrs:
    tmp = 'store_yr_'+str(i)
    filepath = '/Users/Deepthi/Documents/DSE/Q3_DSE220 - Machine Learning/IRI Data - Project/Data'+'/Year'+str(i)+'/saltsnck/'
    filename = os.listdir(filepath)
    file1 = filter(lambda filename: filename.startswith("saltsnck_drug"),filename)
    store_data["year"+str(i)+'drug'] = pd.read_fwf(filepath+'/'+file1[0])
    store_data["year"+str(i)+'drug']['yr'] = str(i)
    store_data["year"+str(i)+'drug']['store_type'] = 'drug'
    file2 = filter(lambda filename: filename.startswith("saltsnck_g"),filename)
    store_data["year"+str(i)+'groc'] = pd.read_fwf(filepath+'/'+file2[0])
    store_data["year"+str(i)+'groc']['yr'] = str(i)
    store_data["year"+str(i)+'groc']['store_type'] = 'groc'

In [4]:
store_data.keys()

['year11drug', 'year11groc']

##### Product attributes

The improved file format, which incorporates further information, is prod_category.xls, 16 There are three sets of files.<br /> 
1. The first set of files are applicable to years 1-6 and are provided in a directory called “parsed stub files”.<br /> 
2. The second set of files are applicable to year 7 and are provided in a directory called “parsed stub files 2007”.<br /> 
3. The third set of files are applicable to years 8-11 are are provided in a directory called “parsed stub files 2008-2011”

In [5]:
# Get the product category details for the analysis

df1 = pd.ExcelFile('parsed stub files/prod_saltsnck.xls')
prod_cat_1 = df1.parse('Sheet1')

df2 = pd.ExcelFile('parsed stub files 2007/prod_saltsnck.xlsx')
prod_cat_2 = df2.parse('Sheet1')

# We will be using only prod_cat_3 for this analysis
df2 = pd.ExcelFile('parsed stub files 2008-2011/prod11_saltsnck.xlsx')
prod_cat_3 = df2.parse('Sheet1')

##### Q1) What are the most popular item(s) for a chosen product/category?

I looked mostly at the number of units sold in the year 2011 as a proxy for popularity of the product. 

REGULAR POTATO CHIP IN A BAG is the most popular item.

Couldn't look into FAT CONTENT, COOKING METHOD, SALT/SODIUM CONTENT or TYPE OF CUT because of missing data.

In [6]:
# For this we will consider the most recent year 2011.

# Combine data for drugstores and grocery stores
frames = [store_data['year11drug'], store_data['year11groc']]
data_2011 = pd.concat(frames)

In [7]:
# Merge the store data with product category data
data_2011_with_cat = pd.merge(data_2011, prod_cat_3, how='left', on=['ITEM', 'ITEM'])

In [38]:
# Get the most popular product 
# Filtered out only required columns as using the entire table was causing the system to crash
temp = data_2011_with_cat[['PRODUCT TYPE','PACKAGE','FLAVOR/SCENT','FAT CONTENT','COOKING METHOD','SALT/SODIUM CONTENT','TYPE OF CUT','UNITS','DOLLARS']]
temp.groupby(['PRODUCT TYPE'])['UNITS','DOLLARS'].sum().sort(['UNITS','DOLLARS'],ascending = [0,0]).head(5)

,UNITS,DOLLARS
PRODUCT TYPE,,
POTATO CHIP,233266784,5.272667e+08
TORTILLA CHIP,121969958,3.131993e+08
RTE POPCORN,109442905,2.471672e+08
PRETZEL,56451920,1.321804e+08
CHEESE SNACK,42998080,8.853130e+07


In [43]:
# Now that we know that 'Potato Chip' is the most popular product, I wanted to understand if we see any preference in packaging
temp[temp['PRODUCT TYPE']=='POTATO CHIP'].groupby(['PACKAGE'])['UNITS','DOLLARS'].sum().sort(['UNITS','DOLLARS'],ascending = [0,0]).head(5)

,UNITS,DOLLARS
PACKAGE,,
BAG,223955809,5.038678e+08
BAG IN BOX,3110279,8.451179e+06
BOX,2838465,7.095764e+06
BAG IN BAG,813948,2.321228e+06
BAG IN BAG INDV WRAP,711263,1.707785e+06


In [51]:
# Knowing that 'Potato chip' in a bag is the most preferred product, I wanted to see if there is any preference for flavour
temp[(temp['PRODUCT TYPE']=='POTATO CHIP')&(temp['PACKAGE']=='BAG')].groupby(['FLAVOR/SCENT'])['UNITS','DOLLARS'].sum().sort(['UNITS','DOLLARS'],ascending = [0,0]).head(5)

,UNITS,DOLLARS
FLAVOR/SCENT,,
REGULAR,65540217,1.463151e+08
CLASSIC,22288386,5.115768e+07
BARBECUE,19855618,4.056266e+07
ORIGINAL,16790944,4.480575e+07
SOUR CREAM & ONION,14981454,3.142769e+07


##### Q2) How does the popularity of an item change with the cost associated with it for a chosen year?

To understand how popularity of an item changes with cost associated with it, I took the data at the most granular level, SKU-week-store level and calculated the cost per unit. Then calculated the correlation of units and cost.

In this exercise, I found that popularity decreases as the cost associated with the product increases.

In [56]:
## Get Dollar/Unit at SKU-week-store level
temp_with_dol_per_unit = data_2011_with_cat[['IRI_KEY','WEEK','L9','UNITS','DOLLARS']].copy()
temp_with_dol_per_unit['DOL_PER_UNIT'] = temp_with_dol_per_unit['DOLLARS']/temp_with_dol_per_unit['UNITS']

In [58]:
temp_with_dol_per_unit[['UNITS','DOL_PER_UNIT']].corr()

,UNITS,DOL_PER_UNIT
UNITS,1.000000,-0.141734
DOL_PER_UNIT,-0.141734,1.000000


##### Q3) How is the popularity affected over the years?